In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import os
key=os.getenv("OPENAI_API_KEY")

In [5]:
llm=ChatOpenAI(
    openai_api_key=key,
    model_name="gpt-4o-mini",
    temperature=0.5
)

C:\Users\visha\AppData\Local\Temp\ipykernel_8736\1799399020.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(


In [6]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000158E57B9590>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000158E57C1C90>, model_name='gpt-4o-mini', temperature=0.5, model_kwargs={}, openai_api_key='sk-kHE9R53XVCiP4JozsIm77-RoceVObWhwTM7p8o_QKgT3BlbkFJFJecWXKtANrvvkkxDPJEO4hV5hJkgojmjOONI9180A', openai_proxy='')

In [7]:
# pip install PyPDF2


In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [12]:
quiz_chain=LLMChain(
    llm=llm, 
    prompt=quiz_generation_prompt, 
    output_key="quiz",
    verbose=True)

C:\Users\visha\AppData\Local\Temp\ipykernel_8736\2361320494.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(


In [13]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [15]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

Now connect ths two both chain together using simple sequential chain

In [16]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [61]:
file_path='C:/Users/visha/mcqgen/Machine Learning Tutorial.txt'

In [62]:
 with open(file_path, "r") as file:
        text = file.read()
        print(text) 

Machine Learning Tutorial
Last Updated : 24 Jan, 2025
Machine learning is a subset of Artificial Intelligence (AI) that enables computers to learn from data and make predictions without being explicitly programmed. If you're new to this field, this tutorial will provide a comprehensive understanding of machine learning, its types, algorithms, tools, and practical applications.

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and me

In [63]:
print(text)

Machine Learning Tutorial
Last Updated : 24 Jan, 2025
Machine learning is a subset of Artificial Intelligence (AI) that enables computers to learn from data and make predictions without being explicitly programmed. If you're new to this field, this tutorial will provide a comprehensive understanding of machine learning, its types, algorithms, tools, and practical applications.

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and me

In [72]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [65]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"
Text=text

In [66]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": text,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine Learning Tutorial
Last Updated : 24 Jan, 2025
Machine learning is a subset of Artificial Intelligence (AI) that enables computers to learn from data and make predictions without being explicitly programmed. If you're new to this field, this tutorial will provide a comprehensive understanding of machine learning, its types, algorithms, tools, and practical applications.

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, incl

In [67]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3747
Prompt Tokens:3138
Completion Tokens:609
Total Cost:0.0008360999999999999


In [86]:
response

{'text': 'Machine Learning Tutorial\nLast Updated : 24 Jan, 2025\nMachine learning is a subset of Artificial Intelligence (AI) that enables computers to learn from data and make predictions without being explicitly programmed. If you\'re new to this field, this tutorial will provide a comprehensive understanding of machine learning, its types, algorithms, tools, and practical applications.\n\nMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, ag

In [88]:
quiz1=response.get("quiz")
quiz1

'{"1": {"mcq": "What is machine learning a subset of?", "options": {"a": "Biology", "b": "Physics", "c": "Artificial Intelligence", "d": "Chemistry"}, "correct": "c"}, "2": {"mcq": "Who coined the term \'machine learning\'?", "options": {"a": "Donald Hebb", "b": "Arthur Samuel", "c": "Tom M. Mitchell", "d": "Geoffrey Hinton"}, "correct": "b"}, "3": {"mcq": "What was the purpose of the early \'learning machine\' developed by Raytheon Company?", "options": {"a": "To play chess", "b": "To recognize patterns", "c": "To analyze weather data", "d": "To translate languages"}, "correct": "b"}, "4": {"mcq": "Which field focuses on exploratory data analysis via unsupervised learning?", "options": {"a": "Statistics", "b": "Data Mining", "c": "Computer Vision", "d": "Natural Language Processing"}, "correct": "b"}, "5": {"mcq": "What is the primary goal of modern-day machine learning?", "options": {"a": "To mimic human emotions", "b": "To classify data and make predictions", "c": "To replace human 

In [89]:
# Convert the JSON string to a dictionary
quiz = json.loads(quiz1)

In [101]:
import json
import pandas as pd

# Assuming response is your dictionary
quiz_str = response.get("quiz")

# Convert the JSON string to a dictionary
quiz = json.loads(quiz_str)

# Prepare data for DataFrame
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    choices = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append((mcq, choices, correct))

In [100]:
# quiz

# quiz_table_data = []
# for key, value in quiz.items():
#     mcq = value["mcq"]
#     options = " | ".join(
#         [
#             f"{option}: {option_value}"
#             for option, option_value in value["options"].items()
#         ]
#     )
#     quiz_table_data.append((key, mcq, options))

KeyError: 'mcq'

In [91]:
# for data in quiz_table_data:
#     print(f"Question {data[0]}: {data[1]}")
#     print(f"Options: {data[2]}\n")

Question 1: What is machine learning a subset of?
Options: a: Biology | b: Physics | c: Artificial Intelligence | d: Chemistry

Question 2: Who coined the term 'machine learning'?
Options: a: Donald Hebb | b: Arthur Samuel | c: Tom M. Mitchell | d: Geoffrey Hinton

Question 3: What was the purpose of the early 'learning machine' developed by Raytheon Company?
Options: a: To play chess | b: To recognize patterns | c: To analyze weather data | d: To translate languages

Question 4: Which field focuses on exploratory data analysis via unsupervised learning?
Options: a: Statistics | b: Data Mining | c: Computer Vision | d: Natural Language Processing

Question 5: What is the primary goal of modern-day machine learning?
Options: a: To mimic human emotions | b: To classify data and make predictions | c: To replace human jobs | d: To create new languages



In [102]:
quiz_table_data

[('What is machine learning a subset of?',
  'a: Biology | b: Physics | c: Artificial Intelligence | d: Chemistry',
  'c'),
 ("Who coined the term 'machine learning'?",
  'a: Donald Hebb | b: Arthur Samuel | c: Tom M. Mitchell | d: Geoffrey Hinton',
  'b'),
 ("What was the purpose of the early 'learning machine' developed by Raytheon Company?",
  'a: To play chess | b: To recognize patterns | c: To analyze weather data | d: To translate languages',
  'b'),
 ('Which field focuses on exploratory data analysis via unsupervised learning?',
  'a: Statistics | b: Data Mining | c: Computer Vision | d: Natural Language Processing',
  'b'),
 ('What is the primary goal of modern-day machine learning?',
  'a: To mimic human emotions | b: To classify data and make predictions | c: To replace human jobs | d: To create new languages',
  'b')]

In [98]:
# quiz=pd.DataFrame(quiz_table_data)

In [103]:
# Create DataFrame
df = pd.DataFrame(quiz_table_data, columns=["mcq", "choices", "correct"])

In [104]:
# Save DataFrame to CSV
csv_file = 'quiz_data.csv'
df.to_csv(csv_file, index=False)

In [105]:

from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'01_29_2025_11_26_14'